Using this notebook for working changes before putting them in a python file

In [4]:
import database_utility
import baseball_models
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [2]:
sqlite_file = 'baseball_data.sqlite'
dh = database_utility.DatabaseHelper(sqlite_file, 'baseball_key_joiner.csv')
#This will give a ValueError when using pd.read_json if the path doesn't exist
filepath = r'C:\Users\Chris\Google Drive\Programming\Baseball\Web Scraping\bbref_scraper\bbref_scraper\bbref.jl'

In [3]:
batting_df = dh.calc_batting_fd_score(filepath1 = filepath, start_date = '2016-01-01', end_date = '2016-12-13')
batting_df = batting_df.rename(columns={'game_date_x' : 'game_date', 'home_team_x' : 'home_team'})

Gathering player lookup table. This may take a moment.


In [6]:
pitching_df = dh.calc_pitching_fd_score(filepath)

C:\Users\Chris\Google Drive\Programming\Baseball\database_utility.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  qual_start_df['quality_start'] = 1


In [5]:
selector = baseball_models.BaseballModelSelection()
X, y, tscv = selector.train_test_split(batting_df, num_splits=2)

X['fd_score'] = y

In [6]:
X = X.groupby('player').filter(lambda x: x['game_id'].count() > 50)
y = X['fd_score']

In [7]:
#selector = baseball_models.BaseballModelSelection()
mov_avg = baseball_models.MovingAverageModel()

#X, y, tscv = selector.train_test_split(pitching_df, num_splits = 5)

X_train = X[X['game_date'] <= '2016-09-15']
y_train = y.iloc[:len(X_train)]

X_test = X[X['game_date'] > '2016-09-15']
y_test = y.iloc[len(X_train):]

i = 0
#for train_index, test_index in tscv.split(X):
    #if i < 4:
        #print("TRAIN:", train_index, "TEST:", test_index)
        #X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        #y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        #i += 1

In [8]:
#fit the moving average model with all the data just for fun
mov_avg.fit(X_train, y_train)

C:\Users\Chris\Google Drive\Programming\Baseball\baseball_models.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.X['fd_score'] = y
C:\Users\Chris\Anaconda3\envs\si671\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
predictions = mov_avg.predict(X_test)

In [10]:
mean_absolute_error(y_test, predictions)

8.186256776962825